<h1>2023년도 2학기 연세대학교 "양자컴퓨팅과 시뮬레이션2" 수업 실습</h1>
<h2> 양자 화학 문제를 풀어봅시다! - From 2021 IBM Quantum Africa Challenge</h3>

<h1>Quantum Chemistry for HIV</h1>

<img src="HIV-1_capsid_wikipedia.png"/>


<h2><a id='preface'>서문</a></h2>


**HIV는 전세계적으로 공공보건의 큰 난제중 하나입니다**. HIV에 연관되어 발생하게 되는 질병의 역학은 영양, 의료시설로의 접근성, 교육 및 연구 지원을 포함한 다양한 사회적 문제를 유발합니다. 바이러스의 빠른 변이는 문제를 더 어렵게 만드는 요인 중 하나입니다. 특히 HIV-1-C 및 HIV-2 변종이 주로 아프리카에서 우세합니다. 자금 지원의 차이로 인해 아프리카 변종 치료를 위한 연구는 다른 프로그램에 비해 뒤처지고 있습니다. 2021 Africa Challenge는 이러한 상황에서 아프리카 연구자들은 이러한 불균형을 해결하기 위해 노력하고 있으며 도구 키트에 양자 컴퓨팅과 같은 최신 기술을 사용하는 것을 지역 사회에 촉구 하였습니다.

**퀀텀 컴퓨팅은 약물 설계에서 크게 기여할 것으로 기대되고 있습니다**. 특히 새로운 항레트로바이러스를 설계하기 위해서는 **화학적 시뮬레이션**을 수행하여 항레트로바이러스가 바이러스 단백질과 결합하는지 확인하는 것이 중요합니다. 이러한 시뮬레이션은 고전적인 슈퍼컴퓨터에서는 어렵고 때로는 효과적이지 않은 것으로 악명이 높습니다. 양자 컴퓨터는 더 나은 약물 설계 작업 흐름을 가능하게 하는 더 정확한 시뮬레이션 가능하게 할 것으로 기대되고 있습니다.  

자세히 설명하자면, 항레트로바이러스제는 단백질분해효소라고 불리는 바이러스 단백질과 결합하고 이를 차단하는 약물로, 바이러스 다단백(polyproteins)을 더 작은 단백질로 절단하여 처리할 수 있도록 해줍니다. 이 단백질분해효소는 화학적 가위로 생각될 수 있습니다. 항레트로바이러스제는 가위가 절단하는 능력을 방해하는 끈적한 장애물로 생각될 수 있습니다. 이 단백질분해효소가 차단되면, 이 바이러스는 더 이상 자신을 복제할 수 없습니다.


바이러스 단백질분해효소의 돌연변이는 특정 항레트로바이러스의 결합력을 변화시킵니다. 따라서 돌연변이가 발생하여 항레트로바이러스가 더 이상 잘 결합하지 않을 때는 항레트로바이러스 분자가 다시 강하게 결합하도록 조정하는 것이 필요합니다.

**이 도전의 주요 목표는 가상의 항레트로바이러스 분자가 가상의 바이러스 단백질분해효소와 결합하는지 여부를 탐구하는 것입니다.**


*HIV가 어떻게 감염되고 항레트로바이러스 치료가 어떻게 작동하는지 설명하는 비디오*:

In [ ]:
from IPython.display import display, YouTubeVideo
YouTubeVideo('cSNaBui2IM8')

<h1>기본 연습: 우주에서 가장 단순한 분자의 바닥 상태 에너지 계산</h1>

이 실습을 위해서는 Qiskit Ecosystem 프로젝트 중 하나인 qiskit-nature 패키지가 필요합니다. 다음의 셀을 실행시켜 설치 후 커널을 재시작 해주세요.

In [ ]:
pip install --prefer-binary pyscf -U qiskit-nature qiskit_algorithms


In [ ]:
import qiskit.tools.jupyter
import qiskit_ibm_runtime
import qiskit_nature
import qiskit_algorithms

%qiskit_version_table


In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

<h2><a id='intro'>시작하기에 앞서</a></h2>

HIV 챌린지에서 우리는 임의의 항레트로바이러스 분자가 임의의 단백질분해효소 분자와 결합하여 이를 파괴하는지 조사하는 임무를 맡았습니다. 분자가 멀리 떨어져 있을 때보다 가까이 있을 때(단일 거대분자를 형성할 때) 성공적인 결합은 전체 시스템의 낮은 바닥 상태 에너지에 의해 결정됩니다.

전체 바닥 상태 에너지는 전자와 핵의 배열에 관한 에너지의 합을 말합니다. 핵 에너지는 고전적으로 계산하기 쉽습니다. 양자 컴퓨터가 필요한 것은 전자 분포의 에너지(즉, 분자 스핀 궤도 점유)입니다.

아래에 이어지는 파트에서는 간단한 분자의 바닥 상태를 계산하며 Qiskit과 Qiskit nature 패키지 사용법을 익혀 보도록 합시다.

어떤 특정한 구성에서 분자의 바닥 상태는 핵의 위치와 핵 주위의 전자 분포로 구성됩니다. 핵-핵, 핵-전자, 전자 힘/ 인력과 반발 에너지는 **해밀토니안**이라 불리는 행렬에 나타나게 됩니다. 핵은 전자에 비해 상대적으로 질량이 크기 때문에 전자보다 느린 시간 규모로 이동합니다. 이렇게 하면 계산을 두 부분으로 나눌 수 있습니다: 핵을 놓고 전자 분포를 계산한 다음, 핵을 이동시키고 최소 총 에너지 분포에 도달할 때까지 전자 분포를 다시 계산할 수 있게 됩니다:

<div class="alert alert-block alert-warning">
<b>알고리듬: 바닥 상태 에너지 계산</b>

원자 핵을 위치 시킵니다
    
계산에 사용할 변수가 다 사용되거나 total_energy에 변화가 없을 때까지 반복합니다:
- 전자 바닥 상태를 계산합니다
- total_energy = (nuclei 반발력 + 전자 에너지)
- 원자핵을 이동합니다(그리드 또는 구배에 따라 이동)
총_에너지를 반환합니다
</div>

연습에서는 간단하게 핵 위치를 고정하지만 나중에 챌린지 섹션에서는 항레트로바이러스와 단백질분해효소 분자 사이의 다양한 1차원 분자 간 거리를 허용하며, 이는 결합을 시도하기 위해 단백질분해효소 분자에 접근하는 항레트로바이러스를 모사합니다.

<h2><a id='step_1'> 1단계: 분자의 구조를 정의합니다</a></h2>

연습을 위해 가장 간단한 분자 중 하나인: H$_2$, 수소 가스 분자를 사용하도록 합니다.

<img src="h2.png"/>

*가장 먼저 해야 할 일은 각 핵의 위치를 지정하는 것입니다. 이것은 파이썬 리스트로 정의되는데, 여기서 각 핵은 원자에 해당하는 문자열과 그 3차원 좌표를 포함합니다. 또한 전체 전하량을 지정하는데, 이것은 Qiskit에게 그 전하를 생성하는 데 필요한 전자의 수를 자동으로 계산하도록 해줍니다. 

핵 위치가 고정되면(핵-핵력은 일시적으로 무시함) 양자 컴퓨터에서 계산해야 하는 해밀토니안의 유일한 부분은 상세한 전자-전자 상호작용입니다. 핵-전자와 대략적인 평균 필드 전자-전자 상호작용은 하트리-폭 근사를 통해 고전 컴퓨터에서 *허용된 분자 오비탈*로 사전 계산될 수 있습니다. 이러한 허용된 분자 오비탈과 사전 계산된 중첩을 통해 Qiskit은 상호 작용하는 전자-전자 **Fermion 분자-오비탈 해밀토니안**(두 번째 양자화를 사용)을 자동으로 생성합니다. 분자 오비탈과 중첩 사전 계산은 PySCF와 같은 고전 패키지에 의해 제공되며 소위 *드라이버*를 통해 Qiskit에 연결되며, 특히 PySCFD 드라이버를 사용합니다.

In [ ]:
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

problem = driver.run()

<h2><a id='step_2'>2 단계: 큐비트 해밀토니안 계산</a></h2>

분자의 구조를 드라이버에 정의한 후 풀어야한 문제를 `driver.run()`으로 설정하고 나면, 양자 컴퓨터를 사용해 계산하기 위해 전자(페르미온 동작을 나타내는)를 양자 컴퓨터의 큐비트(스핀 동작과 밀접하게 관련되어 있지만 반드시 anti-symmetric은 아님)에 매핑해야 합니다. 이 매핑은 위의 드라이버와 무관한 일반적인 프로세스입니다.

Fermion을 큐비트에 매핑하는 여러가지 방법이 있으며, [Qubit Mapper 튜토리얼](https://qiskit.org/ecosystem/nature/tutorials/06_qubit_mappers.html)에서 사용가능한 여러가지 방법을 확인할 수 있습니다.

이곳에서는 "The Jordan-Wigner Mapping"을 사용하도록 하겠습니다. Jordan-Wigner 매핑은 하나의 스핀 궤도의 점유를 하나의 큐비트 점유와 매핑하기 때문에 가장 물리적으로 직관적이며 간단한 매핑법입니다.

<img src="https://qiskit.org/ecosystem/nature/_images/jw_mapping.png"/>




In [ ]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

jw_mapper = JordanWignerMapper()

In [ ]:
fermionic_op = problem.hamiltonian.second_q_op()
print(fermionic_op)

qubit_jw_op = jw_mapper.map(fermionic_op)
print(qubit_jw_op)

<h2><a id='step_3'> 3 단계: 기준이될 고전적 결과값</a></h2>

참조 값으로서 NumPyEigen Solver를 사용하여 이 시스템을 고전적으로 계산해 보도록 합시다.

In [ ]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver

np_solver = NumPyMinimumEigensolver()
np_groundstate_solver = GroundStateEigensolver(jw_mapper, np_solver)

np_result = np_groundstate_solver.solve(problem)

In [ ]:
target_energy = np_result.total_energies[0]
print(np_result)

<h2><a id='step_3'> 3 단계: VQE</a></h2>

이제 분자를 양자 컴퓨터로의 매핑하고, 계산 결과에 대한 기준 값을 계산해 두었으니 분자의 바닥 상태를 계산할 양자 알고리듬을 선택해 봅시다. NISQ 시대의 양자 컴퓨터에 적합한 짧은 양자 회로를 사용할 수 있는 VQE 알고리듬을 선택해 보도록 합시다. VQE에 대해서는 강의 와 앞선 시간에 배웠다고 가정하고 이 연습에서는 실질적인 코딩 구현에 집중해 보도록 합시다.

<img src="vqe_method_NB.png"/>

VQE 솔루션을 정의하려면 다음과 같은 세 가지 필수 요소가 필요합니다:

- Estimator 프리미티브: 

- 변분 계산 형식: 여기서는 UCC(Unitary Coupled Cluster) ansatz를 사용합니다(예: [Physical Review A 98.2 (2018): 022322] 참조). 이 방법은 화학 계산의 표준적인 방법이기 때문에 UCC로 VQE를 빠르게 초기화할 수 있도록 이미 Qiskit Nature에 필요한 기능이 모두 들어있습니다. 기본값은 단일 여기와 이중 여기를 모두 사용하는 것입니다. 그러나 여기 유형(S, D, SD)과 다른 매개변수를 선택할 수 있습니다. 또한 UCCSD 변형 형식 앞에 해결하려는 문제에 따라 큐비트의 점유를 초기화하는 HartreeFock 초기 상태를 추가합니다.

- Optimizer: 변형 형태에 사용하는 변수들을 최적화 하는데 사용합니다. 자세한 것은 [관련 문서](https://qiskit.org/ecosystem/algorithms/apidocs/qiskit_algorithms.optimizers.html)를 참고하도록 합시다.

우선 먼저 변분 계산에 사용할 Ansatz를 선언하도록 합시다.


In [ ]:
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
import numpy as np

ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    jw_mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        jw_mapper,
    ),
)
initial_point = np.random.random(ansatz.num_parameters)

ansatz.decompose().decompose().decompose().draw("mpl", style="iqx")


더 짧은 효과적인 Ansatz로는 다음의 `EfficientSU2`를 사용할 수도 있습니다.

In [ ]:
from qiskit.circuit.library import EfficientSU2

su2_ansatz = EfficientSU2(qubit_jw_op.num_qubits)
su2_initial_point = np.random.random(su2_ansatz.num_parameters)
su2_ansatz.decompose().draw("mpl", style="iqx")

In [ ]:
from qiskit.circuit.library import TwoLocal

twolocal_ansatz = TwoLocal(4, 'ry', 'cz')
twolocal_initial_point = np.random.random(twolocal_ansatz.num_parameters)
twolocal_ansatz.decompose().draw("mpl", style="iqx")

VQE의 반복적인 계산 과정에서 사용할 Cost Fuction도 정의해 보도록 합시다.

In [ ]:


def cost_func(params, ansatz, hamiltonian, estimator):
    global energy
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    energy = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return energy

이제 이 함수를 사용할 Call_Back 함수도 정의해 봅시다.

In [ ]:
def build_callback(ansatz, hamiltonian, estimator, callback_dict):
    """Return callback function that uses Estimator instance,
    and stores intermediate values into a dictionary.

    Parameters:
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance
        callback_dict (dict): Mutable dict for storing values

    Returns:
        Callable: Callback function object
    """
    def callback(current_vector):

        """Callback function storing previous solution vector,
        computing the intermediate cost value, and displaying number
        of completed iterations and average time per iteration.

        Values are stored in pre-defined 'callback_dict' dictionary.

        Parameters:
            current_vector (ndarray): Current vector of parameters
                                      returned by optimizer
        """
        # Keep track of the number of iterations
        callback_dict["iters"] += 1
        # Set the prev_vector to the latest one
        callback_dict["prev_vector"] = current_vector
        # Compute the value of the cost function at the current vector
        # This adds an additional function evaluation
        
        current_cost = energy
        #current_cost = (
        #    estimator.run(ansatz, hamiltonian, parameter_values=current_vector).result().values[0]
        #)
        callback_dict["cost_history"].append(current_cost)
        # Print to screen on single line
        print(
            "Iters. done: {} [Current cost: {}]".format(callback_dict["iters"], current_cost),
            end="\r",
            flush=True,
        )

    return callback

In [ ]:
callback_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

이제 VQE 계산을 하기 위한 모든 준비가 완료 되었습니다. 클라우드의 백엔드를 사용하기 전에 먼저 여러분의 로컬 환경에서 계산을 해보도록 합시다.

계산에는 Sci Python의 minimize함수를 하용하며, 아래의 코드에는 `cobyla` 최적화 방식을 사용했습니다. 사용 가능한 여러 옵션은 [이곳](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)에서 확인 가능합니다.


In [ ]:
from qiskit.primitives import Estimator as QiskitEstimator
from scipy.optimize import minimize

global energy

num_params = su2_ansatz.num_parameters
x0 = 2 * np.pi * np.random.random(num_params)

qiskit_estimator = QiskitEstimator()
callback = build_callback(su2_ansatz, qubit_jw_op, qiskit_estimator, callback_dict)

res = minimize(
    cost_func,
    x0,
    args=(su2_ansatz, qubit_jw_op, qiskit_estimator),
    method="cobyla",
    callback=callback,
)

In [ ]:
import matplotlib.pyplot as plt


fig, ax = plt.subplots()
ax.plot(range(callback_dict["iters"] - 1 ), callback_dict["cost_history"])
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost")
plt.draw()

약간 꼼수를 써보도록 합시다 :) 얻어진 파라미터를 클라우드 백엔드의 입력으로 사용해서 계산량을 조금 줄여 봅시다.

In [ ]:
x0 = res.x

<h2><a id='step_4'> 4 단계: 클라우드의 백엔드로 VQE 계산하기</a></h2>


로컬 VQE 알고리즘에서 `qiskit_nature.runtime.VQClient`로 VQE 클래스를 교환하는 것만으로 로컬 VQE 알고리듬이 아닌 Qiskit Runtime을 사용하여 실행되는 VQE를 사용할 수 있습니다.

가장 먼저 service를 설정하도록 합시다.


In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(instance="ibm-q-yonsei/internal/qc-and-s-2h")
service.backends()
#backend = service.backend("ibmq_qasm_simulator")

In [ ]:
backend = service.backend("ibmq_qasm_simulator")

In [ ]:
real_backend = service.least_busy(filters=lambda x:
                               not x.configuration().simulator
                               and x.status().operational==True)
print("least busy backend: ", real_backend)

In [ ]:
from qiskit_ibm_runtime import Session, Estimator, Options

options = Options()
options.execution.shots = 10000
options.optimization_level = 2
options.resilience_level = 1

In [ ]:

session = Session(backend=backend)
estimator = Estimator(session=session, options=options)

callback = build_callback(su2_ansatz, qubit_jw_op, estimator, callback_dict)
res = minimize(
    cost_func,
    x0,
    args=(su2_ansatz, qubit_jw_op, estimator),
    method="cobyla",
    callback=callback,
    options={'maxiter':10},
)

print(res)

session.close()

In [ ]:
x0 = res.x
res.fun

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(callback_dict["iters"]-1), callback_dict["cost_history"])
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost")
plt.draw()

결과에서 살펴 볼 수 있듯이, 클라우드 상의 qasm simulator를 사용해 계산된 바닥 상태 에너지는 1.8378586416577116 하트리로 고전적으로 해결한 값과 거의 같습니다. 
이것으로 분자의 바닥 상태 에너지를 계산하는 과정을 훑어보았으니 본격적으로 챌린지 문제에 도전해 봅시다. 


만약 여러분이 위의 예제를 실제 백엔드에서 실행해보고자 한다면 UCCSD Ansatz보다 상대적으로 작은 회로의 Depth를 지닌 SU2 Ansatz를 사용해서 실행해 보시길 권장합니다. 아래의 코드는 긴 대기시간이나 실행시간을 사용하므로 수업을 마친 후 여유있게 실행해 보세요.

In [ ]:
#real_backend = service.backend("ibm_hanoi") #원하는 백엔드로 바꾸어 보세요

#x0 = res.x #클라우드 계산에서 최적화된 파라미터를 가져옴
callback_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

with Session(backend=real_backend):
    estimator = Estimator(options=options)
    callback = build_callback(su2_ansatz, qubit_jw_op, estimator, callback_dict)
    res = minimize(
        cost_func,
        x0,
        args=(su2_ansatz, qubit_jw_op, estimator),
        method="cobyla",
        callback=callback,
        options={'maxiter':10},
    )


***


<div align=center class="alert alert-block alert-success">
    <h1><a id='challenge'>The HIV Challenge</a></h1>
</div>


## 1단계: the Protease+Anti-retroviral Macromolecule 정의하기

### Protease
실제 단백질분해효소 분자는 각 사슬에 약 100개의 아미노산으로 구성된 2개의 폴리펩티드 사슬로 구성되며(이 2개의 사슬은 서로 접혀 있음), 인접한 쌍은 소위 *펩티드-결합*에 의해 연결됩니다.

<img src="peptide_bond_wikipedia.png" title="Amino Acid bonding"/>


계산을 단순화하기 위해 분자의 O=C-N 부분에 초점을 맞추겠습니다. 또한 분자를 가능한 현실적으로 만들기 위해 충분한 수소 원자를 유지하고 추가하도록 하겠습니다. 흥미롭게도 HCONH$_2$, 포름아미드는 안정한 분자이며,  이온 용매이므로 이온 결합을 실제로 "자릅"니다.


이것이 계산에 사용할 임의의 분자이며, 실제로 사용되는 단백질 분해 효소는 아니지만, 실제 분자에서 영감을 얻은 것입니다:

<img width=50% src="protease.png"/>

```
"O":  (1.1280, 0.2091, 0.0000)
"N": (-1.1878, 0.1791, 0.0000)
"C": (0.0598, -0.3882, 0.0000)
"H": (-1.3085, 1.1864, 0.0001)
"H": (-2.0305, -0.3861, -0.0001)
"H": (-0.0014, -1.4883, -0.0001)
```

이 분자가 HIV 단백질의 복제를 방해하는 가위라고 상상해 봅시다.

<img width=30% src="carpet_scissors_wikipedia_cropped.png"/>

### 항레트로바이러스 (Anti-retroviral)
항레트로바이러스는 단백질분해효소와 결합하여 **절단 메커니즘을 억제/차단하는** 분자입니다. 이 챌린지를 위해 항레트로바이러스 분자의 대역이 될 단일 탄소 원자를 임의로 선택합니다.

<img width=10% src="arv.png"/>

### 거대 분자 (Macromolecule)
떨어져 있던 두 분자가 가까워 지면, 두 분자가 합ㅊ져서 거대 분자를 형성하게 되며 이때 바깥 전자들은 모든 원자들을 둘러싼 분자 궤도를 형성하게 됩니다.

앞서 설명했듯이 전자의 분표를 양자적으로 계산할때는 원자의 위치를 고정하므로, 원자들을 분리해서 배치하여 계산해야 합니다. 이후의 계산을 위해서는 단백질 분해 효소 분자의 위치를 고정한 채로 항 레트로 바이러스의 위치를 직선상에 두고 움직이며 계산하도록 합니다. 이 "차단" 접근 방식은 가위가 절단하는 것을 방해하지만, 만약 분자가 "붙으면" 효과가 있고 HIV의 복제 노력을 성공적으로 방해하는데 성공한 것으로 간주하도록 합시다.




"가위들" 사이로 질소 원자를 향해 항레트로바이러스 분자가 접근하는 상황에 대해 분자를 정의하도록 합시다:

<img width=50% src="arv_approaches_protease.png"/>
 ```
 "C": (-0.1805, 1.3955, 0.0000)
 ```

## Write your answer code here:

아래에 여러분만의 `macromolecule`을 정의해 봅시다.

[힌트] 수소는 아래와 같이 정의했었습니다!

```
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)
```

In [ ]:
## Add your code here
macromolecule = PySCFDriver(
    atom="your code here",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

<h2><a id='refine_step_2'>2. 2 단계: 계산의 크기를 줄여 봅시다</a></h2>



2단계에서는 분자의 해밀토니안을 큐비트에 매핑해 보도록 하겠습니다. 만약 여러분이 정의한 분자의 해밀토니안을 아무 전처리 없이 큐비트에 매핑하고자 한다면 노트북은 멈출것이고 에러를 반환하게 될 것입니다. 분자의 전하값이 0으로 지정되었고 이것은 30 (= 2\*6+7+8+3\*1)개의 전자를 계산해야 함을 의미하게 되는데 전자의 occupation을 고려하는 2차 양자화 후에 이 시스템은 총 60개의 큐비트가 필요한 60개의 스핀 궤도의 문제로 해석되게 됩니다. 60 큐비트를 고전 컴퓨터로 시뮬레이션 하기위해서는 슈퍼컴퓨터 급의 메모리 용량이 필요하고, 아직 IBM Quantum의 백엔드들도 노이즈가 없는 60개의 큐비트를 계산하기에는 충분하지 않기 때문에 계산의 대상이 되는 해밀토니안을 적절한 근사를 사용해 줄여야 합니다. 

이를 위해 `Active Space Reduction`을 사용하는데, 간단히 설명하면 계산에 기여도가 낮은 비활성 전자의 영향을 평균장 포텐셜(mean-field potential)로 취급하여 시스템의 주요 특징은 유지하면서 계산량을 낮추는 방법론입니다. 자세한 설명은 [ https://arxiv.org/abs/2009.01872.](https://arxiv.org/abs/2009.01872.) 논문을 참고하기 바랍니다.

다행히도 Qiskit Nature에는 이 편리한 기능을 자동으로 수행하는 [ActiveSpaceTransformer](https://qiskit.org/ecosystem/nature/stubs/qiskit_nature.second_q.transformers.ActiveSpaceTransformer.html#activespacetransformer)가 내장되어 있습니다. 이 기능을 사용해, 2개의 활성 전자(최외곽전자부터 시작해서 안쪽으로 카운트)에 의한 2개의 활성 궤도만을 고려하도록 하여 계산양을 줄여보도록 합시다. 

우선 아무 근사를 적용하지 않은 원래 분자의 계산 크기를 확인해 봅시다.

In [ ]:
print(problem.molecule)
print(problem.num_particles)
print(problem.num_spatial_orbitals)

이제 `ActiveSpaceTransformer`를 사용해서 문제의 크기를 줄여봅시다.

In [ ]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

as_transformer = ActiveSpaceTransformer(2, 2)

as_problem = as_transformer.transform(problem)
print(as_problem.num_particles)
print(as_problem.num_spatial_orbitals)
print(as_problem.hamiltonian.second_q_op())

다양한 값을 시도해 보면서 Fermionic Operator가 어떻게 변하는지 확인해 보세요. 여러분이 실제로 계산해 보고 싶은 크기를 찾아내어 보세요.

원하는 크기를 결정했다면 앞서 선언한 Jordan-wigner 변환기를 사용해서 Fermionic 연산자를 큐비트에 매핑해 봅시다.

In [ ]:
fermionic_op = as_problem.hamiltonian.second_q_op()
qubit_op = jw_mapper.map(fermionic_op)
print(qubit_op)

<h2> 3단계: 고전 계산을 통해 기준 값 계산하기 및 local Estimator를 사용해서 결과 확인하기</h2>

먼저 앞선 예제와 같이 `NumPyMinimumEigensolver`를 사용해 계산의 기준이 될 값을 먼저 계산해 봅시다. 여러분이 직접 코딩한 후 결과를 저와 비교해 봅시다.

In [ ]:
np_result = np_groundstate_solver.solve(as_problem)
target_energy = np_result.total_energies[0]
print(np_result)

위의 예제를 참고해서 `qiskit.primitives`의 Estimator를 사용해서 VQE 계산을 해봅시다. 회로의 크기를 감안하여 이번에는 `EfficientSU2` Ansatz를 사용해 봅시다.

In [ ]:
#Setting Ansatz and initial point

ansatz = EfficientSU2(qubit_op.num_qubits)
initial_point = np.random.random(ansatz.num_parameters)
ansatz.decompose().draw("mpl", style="iqx")

이제 qiskit.primitives의 Estimator를 사용해 VQE를 실행해서 주어진 분자들의 바닥 상태 에너지를 계산해 봅시다.

In [ ]:
callback_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

In [ ]:
x0  = np.random.random(ansatz.num_parameters)


qiskit_estimator = QiskitEstimator()
callback = build_callback(ansatz, qubit_op, qiskit_estimator, callback_dict)

res = minimize(
    cost_func,
    x0,
    args=(ansatz, qubit_op, qiskit_estimator),
    method="cobyla",
    callback=callback,
)

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(callback_dict["iters"]), callback_dict["cost_history"])
ax.set_xlabel("Iterations")
ax.set_ylabel("Cost")
plt.draw()

<h2> 3-1 단계: 질소와 탄소 분자 사이의 거리를 바꾸어 가면서 바닥 상태 에너지를 plot해 봅시다.</h2>

이제 로컬 시뮬레이터를 사용해 질소와 탄소 분자 사이의 거리를 바꾸어가며 바닥 상태의 에너지를 그려 봅시다. 이를 위해 먼저 두 분자 사이의 거리를 다양하게 바꾼 분자 구조를 리스트로 만들어서 저장해 봅시다.

In [ ]:
geo= [
    "O 1.1280 0.2091 0.0000; N -1.1878 0.1791 0.0000; C 0.0598 -0.3882 0.0000; H -1.3085 1.1864 0.0001; H -2.0305 -0.3861 -0.0001; H -0.0014 -1.4883 -0.0001; C -0.1805 1.3955 0.0000;",
    "O 1.1280 0.2091 0.0000; N -1.1878 0.1791 0.0000; C 0.0598 -0.3882 0.0000; H -1.3085 1.1864 0.0001; H -2.0305 -0.3861 -0.0001; H -0.0014 -1.4883 -0.0001; C -0.4224	1.1033 0.0000;",
    "O 1.1280 0.2091 0.0000; N -1.1878 0.1791 0.0000; C 0.0598 -0.3882 0.0000; H -1.3085 1.1864 0.0001; H -2.0305 -0.3861 -0.0001; H -0.0014 -1.4883 -0.0001; C -0.5500	0.9493 0.0000;",
    "O 1.1280 0.2091 0.0000; N -1.1878 0.1791 0.0000; C 0.0598 -0.3882 0.0000; H -1.3085 1.1864 0.0001; H -2.0305 -0.3861 -0.0001; H -0.0014 -1.4883 -0.0001; C -0.6776	0.7953 0.0000;",
    "O 1.1280 0.2091 0.0000; N -1.1878 0.1791 0.0000; C 0.0598 -0.3882 0.0000; H -1.3085 1.1864 0.0001; H -2.0305 -0.3861 -0.0001; H -0.0014 -1.4883 -0.0001; C -0.8051	0.6412 0.0000;",
    "O 1.1280 0.2091 0.0000; N -1.1878 0.1791 0.0000; C 0.0598 -0.3882 0.0000; H -1.3085 1.1864 0.0001; H -2.0305 -0.3861 -0.0001; H -0.0014 -1.4883 -0.0001; C -0.9327	0.4877 0.0000;",
    "O 1.1280 0.2091 0.0000; N -1.1878 0.1791 0.0000; C 0.0598 -0.3882 0.0000; H -1.3085 1.1864 0.0001; H -2.0305 -0.3861 -0.0001; H -0.0014 -1.4883 -0.0001; C -1.0602	0.3331 0.0000;",
    "O 1.1280 0.2091 0.0000; N -1.1878 0.1791 0.0000; C 0.0598 -0.3882 0.0000; H -1.3085 1.1864 0.0001; H -2.0305 -0.3861 -0.0001; H -0.0014 -1.4883 -0.0001; C -1.3154	0.0251 0.0000;",
    "O 1.1280 0.2091 0.0000; N -1.1878 0.1791 0.0000; C 0.0598 -0.3882 0.0000; H -1.3085 1.1864 0.0001; H -2.0305 -0.3861 -0.0001; H -0.0014 -1.4883 -0.0001; C -1.4429	-0.1290 0.0000;",
]

distance = [1.579329684, 1.2, 1, 0.8, 0.6, 0.4, 0.2, -0.2, -0.4]


아래의 셀에 분자의 구조를 입력으로 받아 바닥 상태를 계산하는 VQE를 위한 함수를 작성해 봅시다. 우선 local estimator를 사용합니다.

In [ ]:
def run_vqe( molecule):
    #step 1: building a molecule
    macromolecule = PySCFDriver( atom=molecule, basis="sto3g", charge=0, spin=0, unit=DistanceUnit.ANGSTROM,)
    problem = macromolecule.run()

    #step2: reduce problem space
    as_transformer = ActiveSpaceTransformer(2, 2)
    as_problem = as_transformer.transform(problem)

    np_result = np_groundstate_solver.solve(as_problem)

    #step3: setting ansatz and mapper
    mapper = JordanWignerMapper()
    fermionic_op = as_problem.hamiltonian.second_q_op()
    qubit_op = mapper.map(fermionic_op)
    
    su2_ansatz = EfficientSU2(qubit_op.num_qubits)
    x0 = np.random.random(su2_ansatz.num_parameters)
    
    callback_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
    }

    qiskit_estimator = QiskitEstimator()
    callback = build_callback(ansatz, qubit_op, qiskit_estimator, callback_dict)

    res = minimize(
        cost_func,
        x0,
        args=(ansatz, qubit_op, qiskit_estimator),
        method="cobyla",
        callback=callback,
    )
    
    
    return res.fun + list(np_result.extracted_transformer_energies.values())[0] + np_result.nuclear_repulsion_energy

<h2> 마지막 질문: Energy Landscape, To bind or not to bind?</a></h2>

이제 마지막 질문에 답할 차례입니다. 이것은 아마 밀리언 달러 값어치의 질문일 것입니다: 우리의 장난스런 항레트로바이러스 분자는 단백질 분해 효소와 결합 할까요?

Q. 계산 결과에서 결합에 해당하는 확실한 최저점이 보이나요?

```
A. 네! 0 근방에서 최저점이 관측 되는 것으로 보아 결합합니다!
B. 네! 무한히 먼 거리에서 최저점이 관측되는 것으로 보아 아주 먼 거리에서 결합합니다!
C. 아니요! 뚜렷한 최저점은 보이지 않지만 바닥상태 에너지가 계속 음수인 것으로 보아 결합하는 것 같습니다!
D. 아니요! 뚜렷한 최저점이 보이지 않기 때문에 결합하지 않는 것 같습니다!
```


In [ ]:
result = []
for i in geo:
    temp = run_vqe(i)
    print(temp)
    result.append(temp)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(distance, result)
plt.ylabel('Hartree')
plt.xlabel('N-C Distance, ANGSTROM')
plt.show()

<h2> 최종 도전! 실제 양자 백엔드에 계산해 보세요! </h2>

위에서 작성한 코드를 사용해서 같은 계산을 실제 양자 백엔드에 해 볼수 있습니다. 

그럼에 있어서의 도전은 다음과 같습니다.
1. 오랜 대기시간
2. 노이즈로 인해 아주 정확한 VQE 값을 얻지 못할 가능성

그럼에도 불구하고 실제 백엔드에서 위의 그래프를 얻어 보고 싶으신가요? 여러분을 위해 백엔드를 예약해 두었습니다. 한번 도전해 보세요!
계산에는 127 큐비트 백엔드를 사용할 수 있습니다! 근사를 사용하지않고 127 큐비트 백엔드에 분자 시스템을 그대로 계산해 볼까요? 도전은 여러분의 몫입니다!

여러분의 실습을 위해 백엔드가 "예약"되어 있습니다. 다음의 셀을 실행해서 예약된 백엔드를 사용해 보세요!

In [ ]:
service = QiskitRuntimeService(instance="ibm-q-yonsei/internal/reservations")
real_backend = service.get_backend('ibmq_kolkata')

https://learning.quantum.ibm.com/tutorial/variational-quantum-eigensolver


#### <a id='qresource'>Quantum Chemistry Supporting materials</a>

*Self-faced Course*
- [*Quantum Computing for Natural Sciences (with IBM Quantum)*](https://open.hpi.de/courses/qc-nature2023)


*Qiskit-Nature Tutorials*
- https://qiskit.org/ecosystem/nature/tutorials/01_electronic_structure.html 
- https://qiskit.org/ecosystem/nature/tutorials/03_ground_state_solvers.html 


*Code References*
- UCCSD : https://qiskit.org/documentation/stubs/qiskit.chemistry.components.variational_forms.UCCSD.html
- ActiveSpaceTransformer: https://qiskit.org/documentation/nature/stubs/qiskit_nature.transformers.second_quantization.electronic.ActiveSpaceTransformer.html?highlight=activespacetransformer#qiskit_nature.transformers.second_quantization.electronic.ActiveSpaceTransformer

Licensing and notes:
- All images used, with gratitude, are listed below with their respective licenses:
  - https://de.wikipedia.org/wiki/Datei:Teppichschere.jpg by CrazyD is licensed under CC BY-SA 3.0
  - https://commons.wikimedia.org/wiki/File:The_structure_of_the_immature_HIV-1_capsid_in_intact_virus_particles.png by MarinaVladivostok is licensed under CC0 1.0
  - https://commons.wikimedia.org/wiki/File:Peptidformationball.svg by YassineMrabet is licensed under the public domain
  - The remaining images are either IBM-owned, or hand-generated by the authors of this notebook.

- HCONH2 (Formamide) co-ordinates kindly provided by the National Library of Medicine:
  - `National Center for Biotechnology Information (2021). PubChem Compound Summary for CID 713, Formamide. https://pubchem.ncbi.nlm.nih.gov/compound/Formamide.`
- For further information about the Pauli exclusion principle:
https://en.wikipedia.org/wiki/Pauli_exclusion_principle
- We would like to thank collaborators, Prof Yasien and Prof Munro from Wits for extensive assistance.
- We would like to thank all the testers and feedback providers for their valuable input.


In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright